In [1]:
!pip uninstall diffusers
!pip install diffusers[torch]==0.10.2
!pip install transformers scipy ftfy --q
!pip install instagrapi
!pip install gibberish
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.1/503.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.4/82.4 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for instagrapi: filename=instagrapi-1.16.25-py3-none-any.whl size=96080 sha256=2170d224bdce22b1daf26672ce7126434ed3f1e57c57b0fbd3143f7c65e8342a
  Stored in directory: /root/.cache/pip/wheels/d1/68/3b/ec27e13

In [2]:
import pandas as pd
from gibberish import Gibberish
import random
import nltk
nltk.download('omw-1.4')
from nltk.corpus import wordnet
import os
import openai
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
from torch import autocast
from PIL import Image
from kaggle_secrets import UserSecretsClient
from instagrapi import Client
from instagrapi.types import Location

[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [3]:
user_secrets = UserSecretsClient()
PASSWORD = user_secrets.get_secret("IG_PW")
USERNAME = user_secrets.get_secret("IG_UN")
OPENAI_API_KEY = user_secrets.get_secret("OPENAI_API_KEY")

In [4]:
some_words = ['excellent', 'amazing', 'great', 'boring', 'fun', 'interesting']
possible_images = [1, 2, 3, 4]

In [5]:

def get_location():
    cities = pd.read_csv('/kaggle/input/worldcities/worldcities.csv')[['city', 'country', 'lat', 'lng']]
    gib = Gibberish()

    lat, lng = None, None
    if random.random() > 0.95:
        location = 0
        location = gib.generate_word()
    elif random.random() > 0.2:
        vals = random.choice(cities.values)
        location = f'{vals[0]}, {vals[1]}'
        lat, lng = vals[2], vals[3]
    else:
        location = random.choice(cities['city'])
    
    return location, lat, lng

def get_adjective(words):
    
    synonyms = []  
    for syn in wordnet.synsets(random.choice(words)):
        for l in syn.lemmas():
            synonyms.append(l.name())
            
    adjective = random.choice(synonyms)
    return adjective

def get_prompt(adjective, location):
    prompt = f'Create an instagram post about my recent {adjective} trip to {location}'
    return prompt

def get_gpt3_post(prompt):
    openai.api_key = OPENAI_API_KEY
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0.9,
    max_tokens=150,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.6,
    stop=[" Human:", " AI:"]
)
    return response['choices'][0]['text']


def get_images(n_images, post_text):
    model_id = "stabilityai/stable-diffusion-2-base"

    # Use the Euler scheduler here instead
    scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
    pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, revision="fp16", torch_dtype=torch.float16)
    pipe = pipe.to("cuda")

    prompt = [post_text] * n_images
    with autocast("cuda"):
        images = pipe(prompt).images    
        
    filenames = []
    for i, image in enumerate(images):
        image.save(f"/kaggle/working/img{i}.jpeg")
        filenames.append(f"/kaggle/working/img{i}.jpeg")

    return filenames
    


def post_to_instagram(filenames, post_text, location_name, lat, lng):
    cl = Client()
    cl.login(USERNAME, PASSWORD)

    if len(filenames) == 1:
        if lat and lng:
            response = cl.photo_upload(filenames[0], post_text, 
                            location=Location(name=location_name, lat=lat, lng=lng))
        else:
            response = cl.photo_upload(filenames[0], post_text)

    else:
        if lat and lng:
            response = cl.album_upload(filenames, post_text, 
                            location=Location(name=location_name, lat=lat, lng=lng))
        else:
            response = cl.album_upload(filenames, post_text)

    return response.dict()

In [6]:
n_images = random.choice(possible_images)
location, lat, lng = get_location()
adjective = get_adjective(some_words)
prompt = get_prompt(adjective, location)
post = get_gpt3_post(prompt)
filenames = get_images(n_images, post)
post = post + " #ottoflaneur"
#post_to_instagram(filenames, post, location, lat, lng)


Downloading:   0%|          | 0.00/308 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/340 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/905 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/607 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

  0%|          | 0/50 [00:00<?, ?it/s]